ECE 8803 Final Project

In [9]:
import numpy as np
import pandas as pd
import os
from dataloader import OCTDataset
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import argparse
import os
import copy
import matplotlib.pyplot as plt



root = os.getcwd()
train = os.path.join(root, "df_prime_train.csv")
test = os.path.join(root, "df_prime_test.csv")



LABELS_SEVERITY = {35: 0,
                   43: 0,
                   47: 1,
                   53: 1,
                   61: 2,
                   65: 2,
                   71: 2,
                   85: 2}


def normalize_array(arr):
    """
    Normalize a 1D NumPy array to have values between 0 and 1.
    """
    arr_min = np.min(arr)
    arr_max = np.max(arr)
    normalized_arr = (arr - arr_min) / (arr_max - arr_min)
    return normalized_arr



'''Obtain Dataset Images (image, label)'''


mean = (.1706)
std = (.2112)
normalize = transforms.Normalize(mean=mean, std=std)

transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    normalize,
])

args = 'C:/Users/jgril/Documents/GitHub/8803_Final_Project'
image_trainset = OCTDataset(args, 'train', transform=transform)
image_testset = OCTDataset(args, 'test', transform=transform)


print("loading data")

images_train = []
labels_train = []

images_o_train = []
labels_o_train = []

for i in range(len(image_trainset)):
    #flattened training dataset
    image,label = image_trainset.__getitem__(i)
    my_image = normalize_array(image.flatten().numpy())
    images_train.append(my_image)
    labels_train.append(label)

    #non-flattened training dataset
    image_o ,label_o = image_trainset.__getitem__(i)
    my_image_o = torch.from_numpy(normalize_array(image_o.numpy()))
    images_o_train.append(my_image_o)
    labels_o_train.append(label_o)

images_test = []
labels_test = []

images_o_test = []
labels_o_test = []

for i in range(len(image_testset)):
    #flattened training dataset
    image,label = image_trainset.__getitem__(i)
    my_image = normalize_array(image.flatten().numpy())
    #print(my_image)
    images_test.append(my_image)
    labels_test.append(label)

    #non-flattened training dataset
    image_o ,label_o = image_trainset.__getitem__(i)
    my_image_o = torch.from_numpy(normalize_array(image_o.numpy()))
    images_o_test.append(my_image_o)
    labels_o_test.append(label_o)

print("data loaded")




'''Meta Train Data'''
meta_trainset = pd.read_csv(train).dropna()
meta_testset = pd.read_csv(test).dropna()

train = os.path.join(root, "df_prime_train.csv")
test = os.path.join(root, "df_prime_test.csv")

meta_X_train = meta_trainset[['Age', 'Gender', 'Race', "Diabetes_Type", "Diabetes_Years", "BMI", "BCVA", "CST", "Leakage_Index"]].to_numpy()
meta_Y_train = meta_trainset[["DRSS"]].to_numpy()
y = []
for i in range(meta_Y_train.shape[0]):
    y.append(LABELS_SEVERITY[int(meta_Y_train[i])])

meta_Y_train = np.array(y)



'''Meta Test Data'''
meta_X_test = meta_testset[['Age', 'Gender', 'Race', "Diabetes_Type", "Diabetes_Years", "BMI", "BCVA", "CST", "Leakage_Index"]].to_numpy()
meta_Y_test = meta_testset[["DRSS"]].to_numpy() 
y = []
for i in range(meta_Y_test.shape[0]):
    y.append(LABELS_SEVERITY[int(meta_Y_test[i])])

Y_test = np.array(y)


#print(meta_X_train)
#print(meta_Y_train)
#print(meta_X_test)
#print(meta_Y_test)

loading data
data loaded


In [13]:
# evaluate dataset
print("Flattened trained dataset")
print(images_train[0])
print(images_train[0].shape)
print(labels_train[0])
print()

print("Flattened test dataset")
print(images_test[0])
print(images_test[0].shape)
print(labels_test[0])
print()

print("Original trained dataset")
print(images_o_train[0])
print(images_o_train[0].shape)
print(labels_o_train[0])
print()

print("Original test dataset")
print(images_o_test[0])
print(images_o_test[0].shape)
print(labels_o_test[0])

Flattened trained dataset
[0.032      0.01999999 0.028      ... 0.         0.         0.        ]
(50176,)
2

Flattened test dataset
[0.032      0.01999999 0.028      ... 0.         0.         0.        ]
(50176,)
2

Original trained dataset
tensor([[[0.0320, 0.0200, 0.0280,  ..., 0.0000, 0.0000, 0.0000],
         [0.1320, 0.0760, 0.0960,  ..., 0.0000, 0.0000, 0.0000],
         [0.0720, 0.0400, 0.0360,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0400, 0.0080, 0.0040,  ..., 0.0000, 0.0000, 0.0000],
         [0.0160, 0.0040, 0.0080,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0040, 0.0080,  ..., 0.0000, 0.0000, 0.0000]]])
torch.Size([1, 224, 224])
2

Original test dataset
tensor([[[0.0320, 0.0200, 0.0280,  ..., 0.0000, 0.0000, 0.0000],
         [0.1320, 0.0760, 0.0960,  ..., 0.0000, 0.0000, 0.0000],
         [0.0720, 0.0400, 0.0360,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0400, 0.0080, 0.0040,  ..., 0.0000, 0.0000, 0.0000],
         [0.0160, 0.0040, 0

Naive Bayes Implementation

In [35]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

#print(type(images_train[0]))
#print(images_train[0])
X_train = images_train
y_train = labels_train

X_test = images_test
y_test = labels_test


# train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# evaluate the classifier on the test data
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.44347063978965817


SVM

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the data

X_train = np.array(images_train).reshape((len(images_train), -1))
y_train = np.array(labels_train)

X_test = np.array(images_test).reshape((len(images_test), -1))
Y_test = np.array(labels_test)
# Split the data into training and testing sets


# Create the SVM model
model = svm.SVC(kernel='linear')

# Train the SVM model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the performance of the SVM model
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

CNN

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

train_batch_size = 100
test_batch_size = 1
image_trainset
trainloader = DataLoader(image_trainset, batch_size=train_batch_size, shuffle=True)

testloader = DataLoader(image_testset, batch_size=test_batch_size, shuffle=True)

In [ ]:
# define network
class MySimpleCNN(torch.nn.Module):
  def __init__(self):
    """Inititalizes the various layers in the network"""
    super(MySimpleCNN, self).__init__()
    ##TODO
    self.cnn1 = nn.Conv2d(1,16, kernel_size = 3, padding=1)
    self.relu = nn.ReLU()
    self.cnn2 = nn.Conv2d(16,32, kernel_size = 3, padding=1)
    self.maxpool = nn.MaxPool2d(kernel_size = 2)
    self.linear = nn.Linear(401408,3)

  def forward(self, x):
    """Processes the input from the dataloaders to return predicted output 
    probability vectors for each example in the batch.
    
    Parameters
    ----------
    x : torch.tensor, shape(batch_size, 1, 8, 8), dtype torch.float
      output from dataloade containing batch_size number of 8 x 8 images as 
      torch tensors
    
    Returns
    -------
    out : torch.tensor, shape (batch_size,10), dtype= torch.float.
      batch_size number of 10-element vectors for each image in the input batch.  
    """
    ##TODO
    x1 = self.relu(self.cnn1(x))
    x2 = self.relu(self.cnn2(x1))
    x3 = self.maxpool(x2)
    out = self.linear(x3.reshape(x.shape[0],-1))

    return out

In [ ]:
## Problem 3 (d)

# perform training 
lr = 1e-3
epochs = 10

# initliaze the network
net = MySimpleCNN()

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

for epoch in range(epochs):

  net.train() # training mode

  for iteration, (x, y) in enumerate(trainloader):
    
    optimizer.zero_grad()

    out = net(x)
    loss = loss_function(out, y)
    
    loss.backward()
    optimizer.step()

    print('Epoch : {} | Training Loss : {:0.4f}'.format(epoch, loss.item()))



Epoch : 0 | Training Loss : 1.0723
Epoch : 0 | Training Loss : 41.4669
Epoch : 0 | Training Loss : 22.8374
Epoch : 0 | Training Loss : 8.9478
Epoch : 0 | Training Loss : 7.8341
Epoch : 0 | Training Loss : 4.3709
Epoch : 0 | Training Loss : 2.2446
Epoch : 0 | Training Loss : 3.2174
Epoch : 0 | Training Loss : 3.2440
Epoch : 0 | Training Loss : 1.6683
Epoch : 0 | Training Loss : 1.6667
Epoch : 0 | Training Loss : 1.5963
Epoch : 0 | Training Loss : 1.9625
Epoch : 0 | Training Loss : 1.6007
Epoch : 0 | Training Loss : 1.2592
Epoch : 0 | Training Loss : 0.9202
Epoch : 0 | Training Loss : 0.9459
Epoch : 0 | Training Loss : 1.2290
Epoch : 0 | Training Loss : 1.2860
Epoch : 0 | Training Loss : 1.1468
Epoch : 0 | Training Loss : 1.0085
Epoch : 0 | Training Loss : 0.9737
Epoch : 0 | Training Loss : 0.9725
Epoch : 0 | Training Loss : 1.1072
Epoch : 0 | Training Loss : 0.9762
Epoch : 0 | Training Loss : 0.9565
Epoch : 0 | Training Loss : 0.9312
Epoch : 0 | Training Loss : 0.9663
Epoch : 0 | Traini

KeyboardInterrupt: 

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 28 * 28, 3)
        
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = self.fc1(x)
        return x
    def train(self, image_trainset, image_testset, lr=0.01, epochs=10, batch_size=32):
        '''
        # Define the data loaders and transformations
        train_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=1),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])
        test_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])
        '''
        #train_dataset = datasets.ImageFolder(root='./train', transform=train_transforms)
        #test_dataset = datasets.ImageFolder(root='./test', transform=test_transforms)
        self.train_loader = torch.utils.data.DataLoader(image_trainset, batch_size=batch_size, shuffle=True)
        self.test_loader = torch.utils.data.DataLoader(image_testset, batch_size=batch_size, shuffle=False)

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Train the model
        model = CNN()
        for epoch in range(epochs): 
            running_loss = 0.0
            for i, data in enumerate(self.train_loader, 0):
                inputs, labels = data
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                if i % 100 == 99:
                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 100))
                    running_loss = 0.0
    def test(self, lr=0.01, epochs=10, batch_size=32):
        # Evaluate the model on the test set
        correct = 0
        total = 0
        with torch.no_grad():
            for data in self.test_loader:
                images, labels = data
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the test images: %d %%' % (
            100 * correct / total))
        

    
model.train(image_trainset, image_testset, lr=0.01, epochs=10, batch_size=32, )





[1,   100] loss: 1.086
[1,   200] loss: 1.087
[1,   300] loss: 1.084
[1,   400] loss: 1.085
[1,   500] loss: 1.086
[1,   600] loss: 1.084
[1,   700] loss: 1.086
[2,   100] loss: 1.086
[2,   200] loss: 1.086
[2,   300] loss: 1.087
[2,   400] loss: 1.085
[2,   500] loss: 1.085
[2,   600] loss: 1.085
[2,   700] loss: 1.085
[3,   100] loss: 1.090
[3,   200] loss: 1.083
[3,   300] loss: 1.083
[3,   400] loss: 1.087
[3,   500] loss: 1.087
[3,   600] loss: 1.085
[3,   700] loss: 1.083
[4,   100] loss: 1.086
[4,   200] loss: 1.084
[4,   300] loss: 1.085
[4,   400] loss: 1.087
[4,   500] loss: 1.084
[4,   600] loss: 1.086
[4,   700] loss: 1.087
[5,   100] loss: 1.086
[5,   200] loss: 1.084
[5,   300] loss: 1.086
[5,   400] loss: 1.084
[5,   500] loss: 1.086
[5,   600] loss: 1.086
[5,   700] loss: 1.086
[6,   100] loss: 1.086
[6,   200] loss: 1.086
[6,   300] loss: 1.086
[6,   400] loss: 1.082
[6,   500] loss: 1.085
[6,   600] loss: 1.086
[6,   700] loss: 1.084
[7,   100] loss: 1.084
[7,   200] 